# Kubios Cloud demo
18.2.2022, Sakari Lukkarinen\
Health, Information and Communication Technology\
Metropolia University of Applied Sciences

## Introduction

The aim of this Notebook is to demonstrate how to ge data from KubiosCloud and show the measurement details. 

The code is based on the Kubioscloud example for authorization and Kubios Cloud API reference (see the References).

### How to use this Notebook

- For USERNAME and PASSWORD use the Kubios HRV application username and password.
- You can use also the same username and password to login into the Kubios Cloud API.
- In addition you need CLIENT_ID to read the data from Kubioscloud.
- Run the code step-by-step.
- Some of the following code snippets needs info/data from the previous steps, so check carefully how the information is extracted from the JSON data-structures.
- At the end you should have RR- or PP-interval data of one of the selected measurements done with Kubios HRV application and graphical presentations how the heart rate is variating from cycle-to-cycle.

### References

- [Kubioscloud example for authorization](https://bitbucket.org/kubios/workspace/snippets/4X95xd/kubioscloud-example-for-authorization-code)
- [Kubios Cloud API reference](https://analysis.kubioscloud.com/v1/portal/documentation/apis.html#kubioscloud-api-reference)

## Setup

In [ ]:
#!/usr/bin/env python3
"""Kubioscloud example for Authorization code grant"""
import base64
import logging
import re
import uuid
from pprint import pprint
import requests
import matplotlib.pyplot as plt
import numpy as np
import urllib

In [ ]:
# Use your Kubios HRV App username and password
USERNAME = "..."
PASSWORD = "..."
CLIENT_ID = "...."

LOGIN_URL = "https://kubioscloud.auth.eu-west-1.amazoncognito.com/login"
TOKEN_URL = "https://kubioscloud.auth.eu-west-1.amazoncognito.com/oauth2/token"
REDIRECT_URI = "https://analysis.kubioscloud.com/v1/portal/login"

USER_AGENT = "Hyte 2022"  # FIXME: Use unique name for your application

## Opening a session and get tokens

In [ ]:
# Logging info
logging.basicConfig(format="%(asctime)-15s [%(levelname)s]: %(message)s")

log = logging.getLogger(__name__)
log.setLevel(logging.INFO)

csrf = str(uuid.uuid4())

# Login data structure
login_data = {
    "client_id": CLIENT_ID,
    "redirect_uri": REDIRECT_URI,
    "username": USERNAME,
    "password": PASSWORD,
    "response_type": "code",
    "access_type": "offline",
    "_csrf": csrf,
}

# Start a session
session = requests.session()

# Open a session
log.info("Authenticating to '%r' with client_id: %r", LOGIN_URL, CLIENT_ID)
login_response = session.post(
    LOGIN_URL,
    data=login_data,
    allow_redirects=False,
    headers={"Cookie": f"XSRF-TOKEN={csrf}", "User-Agent": USER_AGENT},
)
assert (
    login_response.status_code == 302
), f"Status: {login_response.status_code}, Authentication failed."
code = login_response.headers["Location"].split("=")[1]
log.info("Got code: %r", code)

# Exchange tokens
log.info("Exchanging code to tokens")
exch_data = {
    "client_id": CLIENT_ID,
    "code": code,
    "redirect_uri": REDIRECT_URI,
    "grant_type": "authorization_code",
}
exch_response = session.post(
    TOKEN_URL, data=exch_data
)
log.info("Status code %r", exch_response.status_code)
tokens = exch_response.json()

# What are the possible keys?
log.info(tokens.keys())

# When do these tokens expire?
log.info(f"Tokens expires in: {tokens['expires_in']} seconds.")

## Get user info and results

For more details, see:

- [KubiosCloud API reference](https://analysis.kubioscloud.com/v2/portal/documentation/apis.html#kubioscloud-api-reference)
- [Get user information](https://analysis.kubioscloud.com/v2/portal/documentation/apis.html#get-user-information)
- [Get results](https://analysis.kubioscloud.com/v2/portal/documentation/apis.html#get-results)
    - For daily readiness results, see the example at the end "Get results"

In [ ]:
HEADERS = {"Authorization": tokens["id_token"], "User-Agent": USER_AGENT}

BASE_URL = "https://analysis.kubioscloud.com"

GET_USER_INFO = BASE_URL + "/v2/user/self"
GET_RESULT = BASE_URL + "/v2/result/self"
GET_DAILY_READINESS = BASE_URL + "/v2/result/self?types=readiness&daily=yes"

## Return personal information for the currently authenticated user ##
log.info("Get user info")
response = session.get(GET_USER_INFO, headers = HEADERS)
user_info = response.json()

## To list all results for the current user from the last 30 days (all defaults) ##
log.info("Get results")
response = session.get(GET_RESULT, headers = HEADERS)
result = response.json()

## To list only “daily” readiness results ##
log.info("Get daily readiness results")
response = session.get(GET_DAILY_READINESS, headers = HEADERS)
daily_readiness = response.json()

## Results

### List all readiness and stress index results

In [ ]:
readiness = []
created = []
stress_index = []

print('  n  Ready Stress   Timestamp')

for n, r in enumerate(result['results']):
    r_data = r['result']['readiness']
    c_stamp = r['create_timestamp']
    stress_i = r['result']['stress_index']
    readiness.append(r_data)
    created.append(c_stamp)
    stress_index.append(stress_i)
    print(f'{n:3d}  {r_data:5.0f}  {stress_i:5.1f}   {c_stamp}')


### First measurement

In [ ]:
result['results'][0]

### Last measurement

In [ ]:
result['results'][-1]

### Measurement details

In [ ]:
## Get measurement details

## First measurement
N = 0 
MEASURE_ID = result['results'][N]['measure_id']

GET_RESULT_INFORMATION = BASE_URL + "/v2/measure/self/session/" + MEASURE_ID

log.info("Get result information")
response = session.get(GET_RESULT_INFORMATION, headers = HEADERS)
details = response.json()
pprint(details)

## Read the RR/PPI data

In [ ]:
## Read the DATA_URL and 'measured_timestamp'
DATA_URL = details['measure']['channels'][0]['data_url']
measured = details['measure']['measured_timestamp']

## Read the RR/PPI data from DATA_URL
data = urllib.request.urlopen(DATA_URL)
byte = data.read(2)
rr = []

## Convert the binary data to numpy array
while byte:
    rr.append(int.from_bytes(byte, byteorder = "little"))
    byte = data.read(2)
rr = np.array(rr)

### Show the HRV

In [ ]:
## Time is the cumulative sum of rr-data
x = np.cumsum(rr)/1000

plt.plot(x, rr)
plt.xlabel('time (s)')
plt.ylabel('RR (ms)')
plt.title(measured)
plt.grid()
plt.show()

### Show the Heart rate

In [ ]:
## Heart rate is the inverse of rr (ms)
bpm = 60*1000/rr

plt.plot(x, bpm)
plt.xlabel('time (s)')
plt.ylabel('Heart rate (BPM)')
plt.title(measured)
plt.grid()
plt.show()